<a href="https://colab.research.google.com/github/jeremilev/comp472-project/blob/asg2/Asg2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim import similarities
from enum import Enum
import json
from dataclasses import dataclass

In [3]:
GOOGLE_MODEL = "word2vec-google-news-300"

print(api.info(GOOGLE_MODEL))
model = api.load(GOOGLE_MODEL)

{'num_records': 3000000, 'file_size': 1743563840, 'base_dataset': 'Google News (about 100 billion words)', 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/word2vec-google-news-300/__init__.py', 'license': 'not found', 'parameters': {'dimension': 300}, 'description': "Pre-trained vectors trained on a part of the Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. The phrases were obtained using a simple data-driven approach described in 'Distributed Representations of Words and Phrases and their Compositionality' (https://code.google.com/archive/p/word2vec/).", 'read_more': ['https://code.google.com/archive/p/word2vec/', 'https://arxiv.org/abs/1301.3781', 'https://arxiv.org/abs/1310.4546', 'https://www.microsoft.com/en-us/research/publication/linguistic-regularities-in-continuous-space-word-representations/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F189726%2Frvecs.pdf'], 'che

In [4]:
class Label(Enum):
  GUESS = "guess"
  CORRECT = "correct"
  WRONG = "wrong"

@dataclass
class Question:
  question: str
  answer: str
  choices: [str]
  label: Label = Label.GUESS
  guessed_word: str = None

In [5]:
def get_simmilar_word(question: Question) -> Question:

  try:
    suggestions = model.most_similar_cosmul(positive=question.question)
  except(KeyError):
    question.guessed_word=""
    return question

  for suggestion, _ in suggestions:
    if suggestion in question.choices and question.guessed_word == None:
      question.guessed_word = suggestion
    if suggestion in question.choices and question.guessed_word != None:
      if question.guessed_word == question.answer: question.label = Label.CORRECT
      else: question.label = Label.WRONG

  return question

def get_simmilar_list(questions: dict) -> list[Question]:
  for question in questions:
    get_simmilar_word(question)

  return questions

In [7]:
INP_FILENAME = "/content/synonym.json"
OUTP_FILENAME = f"{GOOGLE_MODEL}-details.csv"
EVAL_FILENAME = "analysis.csv"

questions = []
with open(INP_FILENAME) as f:
  data = json.load(f)
  for obj in data:
    questions.append(Question(**obj))

res = get_simmilar_list(questions)
with open(OUTP_FILENAME, "a") as f:
  for question in res:
    if question.guessed_word == None:
      question.guessed_word = ""
    f.write(",".join([question.question,
                      question.answer,
                      question.guessed_word,
                      question.label.value]))
    f.write("\n")

with open(EVAL_FILENAME, "a") as f:
  length = len(model)
  C = len([x.label == Label.CORRECT for x in res])
  guesses = len([x.label == Label.GUESS for x in res])
  V = len(res) - guesses
  if V != 0:
    accuracy = C/V
  else: accuracy = 0

  f.write(",".join([GOOGLE_MODEL,length, C, V, accuracy]))
  f.write("\n")

TypeError: ignored